# Aim 2: Inpatient Mortality

## Import libraries

In [1]:
import pandas as pd

from scipy import stats

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

## Connect to DB

In [2]:
# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
hostname = 'localhost'
port_number = 5434

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, port=port_number, password='mysecretpassword')

# the below statement is prepended to queries to ensure they select from the right schema
# query_schema = 'set search_path to ' + schema_name + ';'

## Univariate Analysis - Table 1

In [3]:
table1_data = []

### Age

In [4]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
DATE_PART('year', AGE(ADM.admittime, PAT.dob)) as "age"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,age
0,21,111970,216859.0,1,87.0
1,38,185910,248910.0,0,75.0
2,61,189535,217135.0,1,55.0
3,62,116009,216609.0,0,68.0
4,64,172056,232593.0,0,26.0
...,...,...,...,...,...
6125,99865,189346,259297.0,1,77.0
6126,99865,189346,284772.0,1,77.0
6127,99912,189380,236183.0,0,84.0
6128,99985,176670,279638.0,0,53.0


In [6]:
patients_he_1 = sepsis_patients[sepsis_patients.hospital_expire_flag==1].age
patients_he_0 = sepsis_patients[sepsis_patients.hospital_expire_flag==0].age
t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
)

table1_data.append(
    [
        'Age (in years)',
        patients_he_1.mean(),
        patients_he_0.mean(),
        t_value,
        p_value
    ]
)
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722]]

### Ethnicity

In [7]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.ethnicity as "ethnicity"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,ethnicity
0,21,111970,216859.0,1,WHITE
1,38,185910,248910.0,0,WHITE
2,61,189535,217135.0,1,WHITE
3,62,116009,216609.0,0,PATIENT DECLINED TO ANSWER
4,64,172056,232593.0,0,BLACK/AFRICAN AMERICAN
...,...,...,...,...,...
6125,99865,189346,259297.0,1,WHITE
6126,99865,189346,284772.0,1,WHITE
6127,99912,189380,236183.0,0,WHITE
6128,99985,176670,279638.0,0,WHITE


In [8]:
def categorize_ethnicity(ethnicity):
    if 'ASIAN' in ethnicity:
        ethnicity = 'ASIAN'
    elif 'WHITE' in ethnicity:
        ethnicity = 'WHITE'
    elif 'HISPANIC' in ethnicity:
        ethnicity = 'HISPANIC/LATINO'
    elif 'BLACK' in ethnicity:
        ethnicity = 'BLACK'
    else:
        ethnicity = 'OTHER'
    return ethnicity

sepsis_patients['ethnicity'] = sepsis_patients.ethnicity.apply(categorize_ethnicity)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,ethnicity
0,21,111970,216859.0,1,WHITE
1,38,185910,248910.0,0,WHITE
2,61,189535,217135.0,1,WHITE
3,62,116009,216609.0,0,OTHER
4,64,172056,232593.0,0,BLACK
...,...,...,...,...,...
6125,99865,189346,259297.0,1,WHITE
6126,99865,189346,284772.0,1,WHITE
6127,99912,189380,236183.0,0,WHITE
6128,99985,176670,279638.0,0,WHITE


In [9]:
for ethnicity in ['ASIAN', 'WHITE', 'HISPANIC/LATINO', 'BLACK', 'OTHER']:
    patients_he_1 = [1 if (e == ethnicity) else 0 for e in sepsis_patients[sepsis_patients.hospital_expire_flag==1].ethnicity]
    patients_he_0 = [1 if (e == ethnicity) else 0 for e in sepsis_patients[sepsis_patients.hospital_expire_flag==0].ethnicity]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Ethnicity = ' + str(ethnicity),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08]]

### First Care Unit

In [11]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ICU.first_careunit as "first_careunit"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,first_careunit
0,21,111970,216859.0,1,MICU
1,38,185910,248910.0,0,CCU
2,61,189535,217135.0,1,MICU
3,62,116009,216609.0,0,CCU
4,64,172056,232593.0,0,MICU
...,...,...,...,...,...
6125,99865,189346,259297.0,1,SICU
6126,99865,189346,284772.0,1,SICU
6127,99912,189380,236183.0,0,MICU
6128,99985,176670,279638.0,0,MICU


In [12]:
for careunit in ['CCU', 'MICU', 'CSRU', 'SICU']:
    patients_he_1 = [1 if (fcu == careunit) else 0 for fcu in sepsis_patients[sepsis_patients.hospital_expire_flag==1].first_careunit]
    patients_he_0 = [1 if (fcu == careunit) else 0 for fcu in sepsis_patients[sepsis_patients.hospital_expire_flag==0].first_careunit]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Careunit = ' + str(careunit),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903]]

### Marital Status

In [15]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.marital_status as "marital_status"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,marital_status
0,21,111970,216859.0,1,MARRIED
1,38,185910,248910.0,0,WIDOWED
2,61,189535,217135.0,1,MARRIED
3,62,116009,216609.0,0,MARRIED
4,64,172056,232593.0,0,SINGLE
...,...,...,...,...,...
6125,99865,189346,259297.0,1,DIVORCED
6126,99865,189346,284772.0,1,DIVORCED
6127,99912,189380,236183.0,0,MARRIED
6128,99985,176670,279638.0,0,MARRIED


In [16]:
def categorize_maritalstatus(maritalstatus):
    if maritalstatus:
        if 'DIVORCED' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'SEPARATED' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'SINGLE' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'WIDOWED' in maritalstatus:
            maritalstatus = 'SINGLE/SEPARATED/DIVORCED/WIDOW'
        elif 'LIFE PARTNER' in maritalstatus:
            maritalstatus = 'MARRIED'
        elif 'MARRIED' in maritalstatus:
            maritalstatus = 'MARRIED'
        else:
            maritalstatus = 'OTHER'
    else:
        maritalstatus = 'OTHER'
    return maritalstatus

sepsis_patients['marital_status'] = sepsis_patients.marital_status.apply(categorize_maritalstatus)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,marital_status
0,21,111970,216859.0,1,MARRIED
1,38,185910,248910.0,0,SINGLE/SEPARATED/DIVORCED/WIDOW
2,61,189535,217135.0,1,MARRIED
3,62,116009,216609.0,0,MARRIED
4,64,172056,232593.0,0,SINGLE/SEPARATED/DIVORCED/WIDOW
...,...,...,...,...,...
6125,99865,189346,259297.0,1,SINGLE/SEPARATED/DIVORCED/WIDOW
6126,99865,189346,284772.0,1,SINGLE/SEPARATED/DIVORCED/WIDOW
6127,99912,189380,236183.0,0,MARRIED
6128,99985,176670,279638.0,0,MARRIED


In [17]:
for marital_status in ['SINGLE/SEPARATED/DIVORCED/WIDOW', 'MARRIED', 'OTHER']:
    patients_he_1 = [1 if (ms == marital_status) else 0 for ms in sepsis_patients[sepsis_patients.hospital_expire_flag==1].marital_status]
    patients_he_0 = [1 if (ms == marital_status) else 0 for ms in sepsis_patients[sepsis_patients.hospital_expire_flag==0].marital_status]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Maritalstatus = ' + str(marital_status),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


### Insurance

In [19]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.insurance as "insurance"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,insurance
0,21,111970,216859.0,1,Medicare
1,38,185910,248910.0,0,Medicare
2,61,189535,217135.0,1,Private
3,62,116009,216609.0,0,Medicare
4,64,172056,232593.0,0,Medicaid
...,...,...,...,...,...
6125,99865,189346,259297.0,1,Medicare
6126,99865,189346,284772.0,1,Medicare
6127,99912,189380,236183.0,0,Medicare
6128,99985,176670,279638.0,0,Private


In [20]:
def categorize_insurance(insurance):
    if insurance:
        if 'Government' in insurance:
            insurance = 'GOVERNMENT'
        elif 'Private' in insurance:
            insurance = 'PRIVATE'
        elif 'Self Pay' in insurance:
            insurance = 'SELF PAY'
        elif 'Medicare' in insurance:
            insurance = 'MEDICARE/MEDICAID'
        elif 'Medicaid' in insurance:
            insurance = 'MEDICARE/MEDICAID'
        else:
            insurance = 'OTHER'
    else:
        insurance = 'OTHER'
    return insurance

sepsis_patients['insurance'] = sepsis_patients.insurance.apply(categorize_insurance)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,insurance
0,21,111970,216859.0,1,MEDICARE/MEDICAID
1,38,185910,248910.0,0,MEDICARE/MEDICAID
2,61,189535,217135.0,1,PRIVATE
3,62,116009,216609.0,0,MEDICARE/MEDICAID
4,64,172056,232593.0,0,MEDICARE/MEDICAID
...,...,...,...,...,...
6125,99865,189346,259297.0,1,MEDICARE/MEDICAID
6126,99865,189346,284772.0,1,MEDICARE/MEDICAID
6127,99912,189380,236183.0,0,MEDICARE/MEDICAID
6128,99985,176670,279638.0,0,PRIVATE


In [21]:
for insurance in ['MEDICARE/MEDICAID', 'PRIVATE', 'GOVERNMENT', 'SELF PAY']:
    patients_he_1 = [1 if (i == insurance) else 0 for i in sepsis_patients[sepsis_patients.hospital_expire_flag==1].insurance]
    patients_he_0 = [1 if (i == insurance) else 0 for i in sepsis_patients[sepsis_patients.hospital_expire_flag==0].insurance]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Insurance = ' + str(insurance),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


### Admission Type

In [22]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.admission_type as "admission_type"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,admission_type
0,21,111970,216859.0,1,EMERGENCY
1,38,185910,248910.0,0,EMERGENCY
2,61,189535,217135.0,1,EMERGENCY
3,62,116009,216609.0,0,EMERGENCY
4,64,172056,232593.0,0,EMERGENCY
...,...,...,...,...,...
6125,99865,189346,259297.0,1,ELECTIVE
6126,99865,189346,284772.0,1,ELECTIVE
6127,99912,189380,236183.0,0,EMERGENCY
6128,99985,176670,279638.0,0,EMERGENCY


In [23]:
for admission_type in ['URGENT', 'ELECTIVE', 'EMERGENCY']:
    patients_he_1 = [1 if (at == admission_type) else 0 for at in sepsis_patients[sepsis_patients.hospital_expire_flag==1].admission_type]
    patients_he_0 = [1 if (at == admission_type) else 0 for at in sepsis_patients[sepsis_patients.hospital_expire_flag==0].admission_type]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Admission Type = ' + str(admission_type),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


### Admission Location

In [24]:
sepsis_query = \
"""
SELECT
DISTINCT
PAT.subject_id,
ADM.hadm_id,
ICU.icustay_id,
ADM.hospital_expire_flag,
ADM.admission_location as "admission_location"
FROM patients PAT
INNER JOIN admissions ADM
ON PAT.subject_id = ADM.subject_id
LEFT JOIN icustays ICU
ON ICU.subject_id = ADM.subject_id AND ICU.hadm_id = ADM.hadm_id
INNER JOIN diagnoses_icd DIA_ICD
ON DIA_ICD.subject_id = ADM.subject_id AND DIA_ICD.hadm_id = ADM.hadm_id AND DIA_ICD.icd9_code in ('99591', '99592', '78552')
WHERE DATE_PART('year', AGE(ADM.admittime, PAT.dob)) > 18
;
"""

sepsis_patients = pd.read_sql_query(sepsis_query, con)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,admission_location
0,21,111970,216859.0,1,EMERGENCY ROOM ADMIT
1,38,185910,248910.0,0,TRANSFER FROM HOSP/EXTRAM
2,61,189535,217135.0,1,CLINIC REFERRAL/PREMATURE
3,62,116009,216609.0,0,EMERGENCY ROOM ADMIT
4,64,172056,232593.0,0,EMERGENCY ROOM ADMIT
...,...,...,...,...,...
6125,99865,189346,259297.0,1,PHYS REFERRAL/NORMAL DELI
6126,99865,189346,284772.0,1,PHYS REFERRAL/NORMAL DELI
6127,99912,189380,236183.0,0,EMERGENCY ROOM ADMIT
6128,99985,176670,279638.0,0,EMERGENCY ROOM ADMIT


In [25]:
def categorize_admission_location(admission_location):
    if admission_location:
        if 'REFERRAL' in admission_location:
            admission_location = 'REFERRAL'
        elif 'EMERGENCY' in admission_location:
            admission_location = 'EMERGENCY'
        elif 'TRANSFER' in admission_location:
            admission_location = 'TRANSFER'
        elif 'TRSF' in admission_location:
            admission_location = 'TRANSFER'
        else:
            admission_location = 'OTHER'
    else:
        admission_location = 'OTHER'
    return admission_location

sepsis_patients['admission_location'] = sepsis_patients.admission_location.apply(categorize_admission_location)
sepsis_patients

,subject_id,hadm_id,icustay_id,hospital_expire_flag,admission_location
0,21,111970,216859.0,1,EMERGENCY
1,38,185910,248910.0,0,TRANSFER
2,61,189535,217135.0,1,REFERRAL
3,62,116009,216609.0,0,EMERGENCY
4,64,172056,232593.0,0,EMERGENCY
...,...,...,...,...,...
6125,99865,189346,259297.0,1,REFERRAL
6126,99865,189346,284772.0,1,REFERRAL
6127,99912,189380,236183.0,0,EMERGENCY
6128,99985,176670,279638.0,0,EMERGENCY


In [26]:
for admission_location in ['REFERRAL', 'EMERGENCY', 'TRANSFER']:
    patients_he_1 = [1 if (al == admission_location) else 0 for al in sepsis_patients[sepsis_patients.hospital_expire_flag==1].admission_location]
    patients_he_0 = [1 if (al == admission_location) else 0 for al in sepsis_patients[sepsis_patients.hospital_expire_flag==0].admission_location]
    t_value, p_value = stats.ttest_ind(
        patients_he_1,
        patients_he_0
    )

    table1_data.append(
        [
            'Admission Location = ' + str(admission_location),
            sum(patients_he_1),
            sum(patients_he_0),
            t_value,
            p_value
        ]
    )
table1_data

[['Age (in years)',
  84.91874376869391,
  79.14403491755577,
  3.422264391851681,
  0.0006250811391058722],
 ['Ethnicity = ASIAN', 56, 138, -1.1638473949703363, 0.24453117281890696],
 ['Ethnicity = WHITE', 1460, 3047, -0.9182400239799813, 0.358529357829805],
 ['Ethnicity = HISPANIC/LATINO',
  56,
  143,
  -1.400987264851608,
  0.1612685182515495],
 ['Ethnicity = BLACK', 182, 458, -2.443116508157017, 0.014589070828960967],
 ['Ethnicity = OTHER', 252, 338, 5.450995727524083, 5.204359840241054e-08],
 ['Careunit = CCU', 200, 356, 1.7112908156968956, 0.08707804587029497],
 ['Careunit = MICU', 1305, 2770, -1.6443796349501218, 0.10014910264828243],
 ['Careunit = CSRU', 86, 143, 1.5878573610397497, 0.1123701880566653],
 ['Careunit = SICU', 274, 579, -0.4040687714105425, 0.6861762488200903],
 ['Maritalstatus = SINGLE/SEPARATED/DIVORCED/WIDOW',
  947,
  2093,
  -2.604468178460079,
  0.009223914046205005],
 ['Maritalstatus = MARRIED',
  943,
  1868,
  1.2629915794872069,
  0.20664025917200965],


## Summary

In [27]:
table1_df = pd.DataFrame(table1_data, columns=['Attribute', 'Hospital Expired == 1', 'Hospital Expired == 0', 't_value', 'p_value'])
table1_df

,Attribute,Hospital Expired == 1,Hospital Expired == 0,t_value,p_value
0,Age (in years),84.918744,79.144035,3.422264,6.250811e-04
1,Ethnicity = ASIAN,56.000000,138.000000,-1.163847,2.445312e-01
2,Ethnicity = WHITE,1460.000000,3047.000000,-0.918240,3.585294e-01
3,Ethnicity = HISPANIC/LATINO,56.000000,143.000000,-1.400987,1.612685e-01
4,Ethnicity = BLACK,182.000000,458.000000,-2.443117,1.458907e-02
5,Ethnicity = OTHER,252.000000,338.000000,5.450996,5.204360e-08
6,Careunit = CCU,200.000000,356.000000,1.711291,8.707805e-02
7,Careunit = MICU,1305.000000,2770.000000,-1.644380,1.001491e-01
8,Careunit = CSRU,86.000000,143.000000,1.587857,1.123702e-01
9,Careunit = SICU,274.000000,579.000000,-0.404069,6.861762e-01
